### Make sure you have access to the Jupyter Notebook / .py script you used to minimise the risk of your portfolio. You can either use the data we've worked with in the course, or use your own data, for this assignment.

In [1]:
# import libraries
import pandas as pd # for data management and analysis
import numpy as np # for numerical computations
from scipy.optimize import minimize # for optimisation framework

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/kpace1111/portfoliomanagement/main/15stocks_price.csv') # import stock price data

df['date_gsheets'] = pd.to_datetime(df['date_gsheets']) # convert dates as timestamp
df.set_index('date_gsheets', inplace=True) # set date as index

returns_df = df.pct_change(1).dropna() #calc returns of all 10 stocks
#returns_df

In [4]:
def getPortRisk(weights):
    
    '''Returns the annualised standard deviation of a k asset portfolio.'''

    returns_df = df.pct_change(1).dropna()  # estimate returns for each asset
    num_stocks = len(returns_df.columns)  # number of stocks based on number of columns (excluding index col)
                                          # this is a local variable
        
    vcv = returns_df.cov()  # being the variance covariance matrix
    
    var_p = np.dot(np.transpose(weights), np.dot(vcv, weights))  # variance of the multi-asset portfolio
    sd_p = np.sqrt(var_p)  # standard deviation of the multi-asset portfolio
    sd_p_annual = sd_p * np.sqrt(250)  # annualised standard deviation of the multi-asset portfolio
    
    return sd_p_annual

In [5]:
num_stocks = len(df.columns)  # being the number of stocks (this is a 'global' variable)
init_weights = [1 / num_stocks] * num_stocks  # initialise weights (x0)

# Constraint that weights in any asset j must be between 0 and 1 inclusive
bounds = tuple((0, 1) for i in range(num_stocks))

# Constraint that the sum of the weights of all assets must equate to 1
cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1})

results = minimize(fun=getPortRisk, x0=init_weights, bounds=bounds, constraints=cons)
results

     fun: 0.10593479369427394
     jac: array([0.10597393, 0.10585235, 0.10600164, 0.1058007 , 0.10597585,
       0.10618785, 0.10613558, 0.1058246 , 0.10587699, 0.11916338,
       0.11288869, 0.10579355, 0.10934387, 0.1058938 , 0.10634915])
 message: 'Optimization terminated successfully'
    nfev: 176
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([4.79798174e-02, 2.80742506e-01, 7.53106142e-03, 1.29294762e-01,
       3.54257601e-02, 8.40457996e-02, 1.62830961e-01, 1.72868279e-01,
       2.00872906e-02, 0.00000000e+00, 1.57209315e-18, 3.55671962e-03,
       0.00000000e+00, 4.31645298e-02, 1.24725142e-02])

In [6]:
# Check total risk of the equal weighted portfolio
getPortRisk(init_weights)

0.13290599867193698

In [12]:
# Explore optimised weights
optimised_weights = pd.DataFrame(results['x'])
optimised_weights.index = df.columns
optimised_weights.rename(columns={optimised_weights.columns[0] : 'weights'}, inplace=True)
optimised_weights['weights_rounded'] = optimised_weights['weights'].apply(lambda x : round(x, 3))

# 1. Identify any "counterintuitive" weight allocations obtained while minimising the risk of your portfolio.

As an example, while working with our data, allocating weights to Netflix (NFLX) instead of Tesla (TSLA) when minimising the risk of the portfolio seems counterintuitive since NFLX is riskier than TSLA. The same goes for allocating weights to Apple (AAPL) instead of Microsoft (MSFT).



Regardless of whether you're working with the course data or your own, list ALL of the "counterintuitive" weight allocations in the following manner:

- Counterintuitive vs Alternative (reason you'd expect to have chosen the alternative instead of the counterintuitive)

Examples:

- NFLX over TSLA (since NFLX is more risky, why invest more in NFLX?)

- AAPL over MFST (since AAPL is more risky, why invest more in AAPL?)

- KO over BRK.B (since they're both equally risky, so why invest more in KO?)

In [13]:
optimised_weights['individual_risks'] = np.std(returns_df) * np.sqrt(250)
optimised_weights

,weights,weights_rounded,individual_risks
AAPL,4.797982e-02,0.048,0.248003
KO,2.807425e-01,0.281,0.137257
NFLX,7.531061e-03,0.008,0.515302
BRK.B,1.292948e-01,0.129,0.141030
DIS,3.542576e-02,0.035,0.181313
IBM,8.404580e-02,0.084,0.181705
VZ,1.628310e-01,0.163,0.160220
WMT,1.728683e-01,0.173,0.169281
GE,2.008729e-02,0.020,0.184478
TSLA,0.000000e+00,0.000,0.482230


Counterintuitive allocations:
- WMT has a higher weighting than VZ, BRK.B, yet WMT is riskier than both
- VZ has a higher weighting than BRK.B, yet VZ is riskier than BRK.B
- AAPL has a higher weighting than UN, DIS, GE, V, MA, MSFT, yet is riskier than all of them
- NFLX has a higher weighting than AMZN, TSLA, MA, MSFT, yet is riskier than all of them

## 2. Without looking at the correlations of the securities, comment on what you expect the correlations of "counterintuitive" stocks to be relative to the alternatives.

For example, it's counterintuitive that we chose NFLX over TSLA when minimising the risk of the portfolio. Do you think the correlation of NFLX with all the other securities is weaker, stronger, or as strong as the correlation of TSLA with all the other securities? If so, why? If not, why not?

- KO must have the lowest correlations to all other assets in the portfolio
- WMT must have a lower correlation to the other assets than VZ and BRK.B
- VZ must have a lower correlation to the other assets than BRK.B
- AAPL must have a lower correlation to the other assets than UN, DIS, GE, V, MA, MSFT
- NFLX must have a lower correlation to the other assets than  AMZN, TSLA, MA, MSFT

## 3. Create a correlation matrix which displays the correlations across all the securities you're working with. Modify the correlation matrix so it only displays data in the lower diagonal, with all values rounded off to 2 decimal places.

In [19]:
def get_correlation_matrix():
     return np.round(returns_df.corr(), 2)
 
 
get_correlation_matrix()

,AAPL,KO,NFLX,BRK.B,DIS,IBM,VZ,WMT,GE,TSLA,MA,AMZN,MSFT,UN,V
AAPL,1.00,0.21,0.12,0.31,0.27,0.26,0.17,0.17,0.26,0.20,0.35,0.26,0.33,0.25,0.30
KO,0.21,1.00,0.09,0.47,0.37,0.32,0.37,0.31,0.35,0.14,0.36,0.23,0.32,0.41,0.33
NFLX,0.12,0.09,1.00,0.19,0.15,0.12,0.04,0.10,0.16,0.23,0.25,0.30,0.21,0.15,0.22
BRK.B,0.31,0.47,0.19,1.00,0.54,0.48,0.42,0.33,0.55,0.22,0.53,0.33,0.43,0.42,0.50
DIS,0.27,0.37,0.15,0.54,1.00,0.34,0.34,0.27,0.43,0.23,0.43,0.31,0.35,0.33,0.41
IBM,0.26,0.32,0.12,0.48,0.34,1.00,0.30,0.23,0.43,0.18,0.38,0.23,0.39,0.30,0.36
VZ,0.17,0.37,0.04,0.42,0.34,0.30,1.00,0.29,0.37,0.11,0.28,0.17,0.29,0.31,0.27
WMT,0.17,0.31,0.10,0.33,0.27,0.23,0.29,1.00,0.24,0.11,0.25,0.14,0.22,0.23,0.25
GE,0.26,0.35,0.16,0.55,0.43,0.43,0.37,0.24,1.00,0.19,0.40,0.26,0.34,0.33,0.38
TSLA,0.20,0.14,0.23,0.22,0.23,0.18,0.11,0.11,0.19,1.00,0.27,0.26,0.21,0.18,0.23


# 4. Explore the correlations of the "counterintuitive" stocks against all other securities. Compare those correlations to those of the "alternative" securities. What do you see?

For example, it's counterintuitive that we chose NFLX over TSLA when minimising the risk of the portfolio. Explore the correlation of NFLX with all other securities. Now compare those correlations with the correlation of TSLA with all other securities.

What do you see?

The "counterintuitive" stocks have weaker relationships with the other securities int he portfolios, relative to the "alternative" securities, whose correlations with other securities is likely more positive and stronger.

- KO has the lowest correlations to all other assets in the portfolio
- WMT has a lower correlation to the other assets than VZ and BRK.B
- VZ has a lower correlation to the other assets than BRK.B
- AAPL has a lower correlation to the other assets than UN, DIS, GE, V, MA, MSFT
- NFLX has a lower correlation to the other assets than  AMZN, TSLA, MA, MSFT

# 5. For each of the "counterintuitive" weight allocations you listed in Question 1, explain why those seemingly counterintuitive results are actually very intuitive. Explain why those weights were allocated; so why riskier stocks were chosen over less risky stocks when minimising the risk of the portfolio.

Explain why more money was invested in one stock over the other, despite both stocks being approximately equally risky.

The main (and same) reason for all of the counterintuitive assets is because of their correlation. while we are minimizing the risk of the portfolio it tries to allocate weights to assets with a lower correlation like KO in our case. While the stocks may be riskier on their own, their relationships between other securities in the portfolio mean that they can decrease the total risk of the portfolio more, compared to the less risky stocks with stronger correlations.